In [1]:
import os
from os import path
from glob import glob
import shutil
from ast import literal_eval

import numpy as np
from tqdm.auto import tqdm

from allensdk.brain_observatory.ecephys.lfp_subsampling import subsampling

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [2]:
source_path = '../tiny-blue-dot/zap-n-zip/EEG_exp/'
dest_path = '../tiny-blue-dot/zap-n-zip/compressed/'
cont_files = [f for f in glob(path.join(source_path, '**'), recursive=True) if 'continuous.dat' in f]
# cont_files += [f for f in glob(path.join(source_path, '**'), recursive=True) if 'continuous-corrected.dat' in f]

In [19]:
# loading data again
eeg_cont_path = path.join(dest_path, 'mouse599975/estim_vis_2022-03-31_12-03-06/experiment1/recording1/continuous/Rhythm_FPGA-101.0/continuous.dat')
path.exists(eeg_cont_path)

True

In [20]:
data = np.load(eeg_cont_path, mmap_mode='r', )

In [21]:
data.shape

(2648576, 40)

In [10]:
data = np.memmap(eeg_cont_path, dtype='int16', mode='r', )
data.shape[0]

105943104

In [14]:
data = np.memmap(eeg_cont_path, dtype='int16', mode='r', )
data.shape[0]/5

105943040.0

In [15]:
data = data.reshape((int(data.size/40), 40))
data.shape

(13242880, 40)

In [16]:
channels = np.arange(40, dtype='int')

In [17]:
data_ss = subsampling.subsample_lfp(data, channels, subsampling_factor=2500//500)

In [18]:
data_ss.shape

(2648576, 40)

In [8]:
# create directory structure identical to EEG_exp folder under compressed
for f in cont_files:
    os.makedirs(path.dirname(f.replace(source_path, dest_path)), exist_ok=True)

In [4]:
# downsample EEG files from 2500 Hz to 500 Hz

for fn in tqdm(cont_files):
    if 'recording' in fn and not path.exists(fn.replace(source_path, dest_path)):
        print(fn.split('EEG_exp/')[1])
        try:
            infofiles = sorted(
                glob(
                    path.dirname(path.dirname(path.dirname(fn))) + '/structure.oebin', recursive=True
                )
            )
            Info = literal_eval(open(infofiles[0]).read())
            Rhythm_info = next((
                sub for sub in Info['continuous'] if sub['source_processor_name'] == 'Rhythm FPGA'
            ), None)
            sample_rate = Rhythm_info['sample_rate']
            num_chs = Rhythm_info['num_channels']
            channels = np.arange(num_chs, dtype='int')

            data = np.memmap(fn, dtype='int16', mode='r')
            data = data.reshape((int(data.size/num_chs), num_chs))

            print(f'sample rate: {sample_rate}, num_chs: {num_chs}, total time: {int(data.size/num_chs)/sample_rate}\n')

            data_ss = subsampling.subsample_lfp(data, channels, subsampling_factor=sample_rate//500)
            np.save(fn.replace(source_path, dest_path), data_ss)
            
            os.rename(
                fn.replace(source_path, dest_path)+'.npy',
                fn.replace(source_path, dest_path)
            )
        except:
            print('Failed\n')

mouse635890/signal_test_2022-08-24_15-46-54/experiment1/recording1/continuous/Acquisition_Board-102.Rhythm Data/continuous.dat
Failed

mouse638703/signal_test_2022-10-03_13-25-48/experiment1/recording1/continuous/Acquisition_Board-105.Rhythm Data/continuous.dat
Failed

mouse547867/rec2_2020-11-20_12-53-00/experiment1/recording1/continuous/Neuropix-PXI-152.0/continuous.dat
Failed

mouse547867/rec2_2020-11-20_12-53-00/experiment1/recording1/continuous/Neuropix-PXI-152.1/continuous.dat
Failed

mouse547867/rec1_2020-11-20_12-42-21/experiment1/recording1/continuous/Neuropix-PXI-152.0/continuous.dat
Failed

mouse547867/rec1_2020-11-20_12-42-21/experiment1/recording1/continuous/Neuropix-PXI-152.1/continuous.dat
Failed

test_NP4/test_v0.5.3_2021-01-20_18-28-06/Record Node 105/experiment1/recording1/continuous/Neuropix-PXI-100.2/continuous.dat
Failed

test_NP4/test_v0.5.3_2021-01-20_18-28-06/Record Node 105/experiment1/recording1/continuous/Neuropix-PXI-100.3/continuous.dat
Failed

test_NP4/tes

In [9]:
# downsample LFP files from 2500 Hz to 500 Hz

for fn in tqdm(cont_files):
    if 'probe' in fn and not path.exists(fn.replace(source_path, dest_path)):
        if '100.0' in fn:
            # skip AP recordings and only downsample LFP band recordings
            continue
        print(fn.split('EEG_exp/')[1], end=', ')
        print(fn[fn.find('probe'):fn.find('probe')+6])
        try:
            sample_rate = 2500
            num_chs = 384
            channels = np.arange(0, num_chs, 4, dtype=int) # alternate rows one channel per row

            data = np.memmap(fn, dtype='int16', mode='r')
            data = np.reshape(data, (int(data.size/num_chs), num_chs))

            print(f'sample rate: {sample_rate}, num_chs: {num_chs}, total time: {int(data.size/num_chs)/sample_rate}\n')

            data_ss = subsampling.subsample_lfp(data, channels, subsampling_factor=sample_rate//500)
            np.save(fn.replace(source_path, dest_path), data_ss)
            
            os.rename(
                fn.replace(source_path, dest_path)+'.npy',
                fn.replace(source_path, dest_path)
            )
        except:
            print('Failed\n')